# Package

In [24]:
import pandas as pd
import numpy as np
#from sklearn.neighbors import NearestNeighbors
from numpy import array
from numpy.linalg import norm

from operator import itemgetter
import timeit

# Functions


In [25]:
def movieId_by_urserId(userid,df):
    """
    Input
    -----
        userid:int
        df:dataframe of train/test rating csv
    Return
    -----
    a list of movie id that rated by given user"""
    return list(df[df['userId'] == userid]['movieId'])

In [27]:
def Average(lst): 
    return sum(lst) / len(lst) 
  

In [82]:
def search_rating_user_id(userid,movieid,time):
    """
    Input
    ---
    userid:int, represent userid
    movieid:int, represent movieid (closest id that rated by given user)
    time:bin_time"""
    return rating[(rating['userId'] == userid) & (rating['movieId'] == movieid) &(rating['bin_num'] == time)]['rating']

In [29]:
def q_given_movieid(df,l_movieid):
    """
    Input
    ----
        df:q df,latent values of movies
        l_movieid:list of movie id, all element should be str 
    Return
    ----
        a dataframe that only contain given movieids"""
    
    return df[l_movieid]

In [30]:
def cosine_similarity (test_movie_id,userid):
    """
    Global variable
    ------
        data_train: train part of rating.csv
        
    Input
    -----
        test_movie_id: the id of movie that want to test,str. for testing purpose should also match same user
        user_id: user id,int
    Return
    
    ----
        """
    #############
    #list of movie id for given user in train,int
    train_movieid_given_user = movieId_by_urserId(userid,data_train)
    #list of movie id for given user in train,str
    train_movieid_given_user_str = [str(val) for val in train_movieid_given_user]
    
    
    ###########
    
    
    l_latent_movie = []
    movie_cos_dic = {}
    for movie in train_movieid_given_user_str:
        #print(movie,type(movie)) #2，str

        numerator =array([val
                      for sublist in np.asarray(q[[test_movie_id]]).tolist()
                      for val in sublist]).dot(array([val
                      for sublist in np.asarray(q[[movie]]).tolist()
                      for val in sublist]))

        cosin_s = numerator/(norm([val
                      for sublist in np.asarray(q[[test_movie_id]]).tolist()
                      for val in sublist])*norm([val
                      for sublist in np.asarray(q[[test_movie_id]]).tolist()
                      for val in sublist]))
        l_latent_movie.append(cosin_s)# cosimilarity value for test latent vs all train latent
        
        dict_movieid_cos = dict(zip(train_movieid_given_user_str,l_latent_movie))
        l_sorted_s_to_l = sorted(dict_movieid_cos.items(), key=itemgetter(1)) #from small to large
        
    return [ele for ele in reversed(l_sorted_s_to_l)] 

In [31]:
def knn_k_user_base(k,test_movie_id,userid,df_mat):
    """
    Input
    -----
        k: int, knn's k
        test_movie_id: the id of movie that want to test,str. for testing purpose should also match same user
        userid: user id,int
        df_mat: matrix after multiple with bias #9724 rows × 610 columns
    Return
    -----
        a list of 23 predicted rating for given movie id for given user 
        
        """
    distance_pair = cosine_similarity(test_movie_id,userid)
    
    l_movie_id = []
    for i in range(k):
        l_movie_id.append(int(distance_pair[i][0])) #str to int
        
#     l_user_id = [userid]*k
    
#     knn_pair = list(zip(l_user_id,l_movie_id))
    
#     for pair in knn_pair:
    l_rate_mean = []
    for i in range(23):
        l_rate = []
        for movie in l_movie_id:
        
            l_rate.append(df_mat[userid][movie][i])
        
        l_rate_mean.append(round(Average(l_rate),1))

    
    return l_rate_mean

In [32]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [46]:
def knn_user_base_campare(test_movie_id,userid, time,df_mat,k = 1):
    knn_result =  knn_k_user_base(k,test_movie_id,userid,df_mat)
    real_rating = search_rating_user_id(userid,int(test_movie_id),time)
    rmse_result = rmse(array(knn_result), real_rating.values)
    return rmse_result

In [ ]:
def knn_campare(test_movie_id,df_mat,time):
    knn_result =  knn_k_(k,test_movie_id,userid,df_mat)
    real_rating = search_rating_user_id(userid,int(test_movie_id),time)
    rmse_result = rmse(array(knn_result), real_rating.values)
    return rmse_result

In [60]:
def cosine_similarity_all(test_movie_id):
    """
    Global variable
    ------
        data_train: train part of rating.csv
        
    Input
    -----
        test_movie_id: the id of movie that want to test,int. for testing purpose should also match same user
    Return
    
    ----
        """
  
    ###########
    movie_test_id_str_list = list(str(test_movie_id))

    movie_compare_id = [item for item in movie_id if item not in movie_test_id_str_list] #movie_id list without test id
    ###############

    l_latent_movie = []
    movie_cos_dic = {}
    for i in movie_compare_id:
       # print(i,type(i)) 2，str

        numerator =array([val
                      for sublist in np.asarray(q[[movie_test_id_str_list[0]]]).tolist()
                      for val in sublist]).dot(array([val
                      for sublist in np.asarray(q[[i]]).tolist()
                      for val in sublist]))

        cosin_s = numerator/(norm([val
                      for sublist in np.asarray(q[[movie_test_id_str_list[0]]]).tolist()
                      for val in sublist])*norm([val
                      for sublist in np.asarray(q[[i]]).tolist()
                      for val in sublist]))
        l_latent_movie.append(cosin_s)


        dict_movieid_cos = dict(zip(movie_compare_id,l_latent_movie))
        l_sorted_s_to_l = sorted(dict_movieid_cos.items(), key=itemgetter(1)) #from small to large

    return ([ele for ele in reversed(l_sorted_s_to_l)])



In [142]:
def mat_check_rate(test_movie_id,df,userid,time):
    
    user_index = b_user_dic.get(str(userid))
    time_index = time - 1
    movie_index =b_item_dic.get(str(test_movie_id))
    rating_predict = df[user_index][movie_index][time_index]
    return round(rating_predict,1)

In [138]:
def knn_k_all(test_movie_id,df,userid,time):
    """
    Input
    -----
        test_movie_id: the id of movie that want to test,str. for testing purpose should also match same user
        df: df_mat
        userid： int
        time: bin_num
    Return
    -----
        a list of  predicted rating for given movie id for given user given year of matrix
        
        """
    distance_pair = cosine_similarity_all(test_movie_id)

    movie_index = b_item_dic.get(distance_pair[0][0])
    user_index = b_user_dic.get(str(userid))
    time_index = time - 1
    rating_predict = df[user_index][movie_index][time_index]
    
    return round(rating_predict,1)

In [136]:
knn_k_all(1,df_mat,1,5)

4.252334262658815

In [143]:
mat_check_rate(1,df_mat,1,5)

4.7

# Data

In [78]:
#result = pyreadr.read_r('../output/mat_fac.RData') 
p = pd.read_csv('../output/p_5.csv',index_col='Unnamed: 0')
#rmse = pd.read_csv('../data/rmse.csv')
q = pd.read_csv('../output/q_5.csv',index_col='Unnamed: 0') #movie latent values
#rating = pd.read_csv('../data/ml-latest-small/ratings.csv')
rating = pd.read_csv('../output/data_bin.csv',index_col = 'Unnamed: 0')
data_train = pd.read_csv('../output/data_train.csv',index_col='Unnamed: 0')
data_test = pd.read_csv('../output/data_test.csv',index_col='Unnamed: 0')

b_item = pd.read_csv('../output/b_item.csv',index_col='Unnamed: 0')
b_time = pd.read_csv('../output/b_time.csv',index_col='Unnamed: 0')
b_user = pd.read_csv('../output/b_user.csv',index_col='Unnamed: 0')

In [133]:
b_item_dic = dict(zip(list(b_item.columns),range(len(b_item.columns)))) #id, value
b_user_dic = dict(zip(list(b_user.columns),range(len(b_user.columns)))) #id, value

# Processing Data

In [34]:

######### processing for test functions
userid = 1

data_test_sort = data_test.copy()
data_test_sort = data_test_sort.sort_values(by=['userId'])

# #list of movie id for given user in train,int
# train_movieid_given_user = movieId_by_urserId(userid,data_train)
#list of movie id for given user in test,int
test_movieid_given_user = movieId_by_urserId(userid,data_test_sort)

#train_movieid_given_user_str = [str(val) for val in train_movieid_given_user]
test_movieid_given_user_str = [str(val) for val in test_movieid_given_user]


In [53]:
movie_id = list(q.columns[:]) #list of all movie index,str'1'
movie_id_int = [int(val) for val in movie_id]

#movie_test_id = 1 #int



In [35]:
#set(data_train['userId']) == set(data_test['userId']) #check, True

In [36]:
#set(rating['userId']) == set(data_test['userId']) #check, True

In [37]:
data_test_sort = data_test.copy()
data_test_sort = data_test_sort.sort_values(by=['userId'])

userId = list(set(rating['userId'])) #a list of user id, int

movie_id = list(q.columns[1:]) #list of all movie id,str

#movie_id_int = [int(val) for val in movie_id] #int form of movie_index
mu = Average(rating['rating'].values)

In [38]:
##calculate predict rating matrix

p_matrix = p.values
q_matrix = q.values
b_item = b_item.values
b_time = b_time.values
b_user = b_user.values


matrix_before_time =  (q_matrix.T.dot(p_matrix))+ mu + b_item.T + b_user
matrix_after_time = matrix_before_time.copy()

matrix_after_time=matrix_after_time.astype(object)
df_mat =  pd.DataFrame(matrix_after_time)


# df_mat.at[0,0]=df_mat[0][0] + b_time.T[0]
# df_mat

In [13]:
# for i in range(610):
#     for j in range(9724):
#         df_mat.at[i,j] = df_mat[i][j] + b_time.T[j]


In [39]:
tic=timeit.default_timer()
for row_index in range(9724):
    for col_index, value in df_mat.iteritems():
    
#         df_mat.at[col_index,row_index] = df_mat[col_index][row_index] + b_time.T[row_index]
        df_mat.set_value(row_index,col_index , df_mat[col_index][row_index] + b_time.T[row_index]) 
    
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


199.57464565499686

In [61]:
cosine_similarity_all(2)

[('2318', 0.9065766310091082),
 ('66915', 0.9038966733419468),
 ('26712', 0.8875288449926366),
 ('27480', 0.8752079505567653),
 ('4064', 0.8748103841596478),
 ('161044', 0.8663286836835109),
 ('7937', 0.8609481150098532),
 ('136778', 0.8579483780725969),
 ('86332', 0.8468099771222983),
 ('88', 0.8461790063484809),
 ('77798', 0.834333750907139),
 ('3786', 0.8262676372728507),
 ('3431', 0.8232211418664159),
 ('1285', 0.8224422288118334),
 ('123947', 0.8212082039738565),
 ('7151', 0.8176714170532196),
 ('921', 0.816754431357703),
 ('127164', 0.81576016904623),
 ('133365', 0.811175408107531),
 ('466', 0.8061803934535826),
 ('3045', 0.8058191231169969),
 ('7324', 0.8041933429526139),
 ('39', 0.8041564839237832),
 ('8199', 0.8015283675297743),
 ('122888', 0.8007386627106003),
 ('3816', 0.7952101202707409),
 ('3536', 0.7949877493007854),
 ('4190', 0.7896890265810578),
 ('58303', 0.7883339363578336),
 ('3835', 0.7873343418667682),
 ('45431', 0.7865072926671436),
 ('5963', 0.7842533644830578),


# scrachwork

In [86]:
a =search_rating_user_id(1,int(test_movieid_given_user_str[0]))

In [47]:
knn_user_base_campare(test_movieid_given_user_str[0],userid,df_mat)

1.6662898124667374

In [98]:
for i in range(30):
    print(knn_user_base_campare(i+1,test_movieid_given_user_str[0],userid,df_mat))

1.6662898124667374
1.4575142627387756
1.247083554259436
1.3126475072390416
1.0778400300762552
0.9236035287182859
0.8091863868699549
0.8737724308248489
0.6981341592758702
0.6662679967387454
0.7407399355873242
0.7465864346127822
0.7404463988468155
0.7885043629776456
0.7567091222208002
0.608276253029822
0.5956363057232595
0.6036123143930685
0.615029161556765
0.5598912937968988
0.48633500709946303
0.48544018572121067
0.5377085035431454
0.6167939547439613
0.570659420243847
0.5348709718443684
0.5421374765483944
0.544138729113945
0.5774757664781833
0.5377085035431453


In [74]:
knn_pair =  knn_k_user_base(15,test_movieid_given_user_str[0],1,df_mat)
knn_pair

[3.2,
 3.3,
 3.4,
 3.3,
 3.2,
 3.1,
 3.3,
 3.5,
 3.3,
 3.5,
 3.2,
 3.5,
 3.4,
 3.2,
 3.3,
 3.1,
 3.1,
 3.0,
 3.1,
 3.1,
 3.2,
 3.2,
 3.4]

In [38]:
df_mat = df_mat.drop([610], axis=1)

In [39]:
df_mat.isnull().values.any()

False

In [41]:
# for col_index, value in df_mat.iteritems():
#     for row_index in range(9724):
#         print(col_index,row_index)

In [ ]:
# for i in range(610):
#     for j in range(9724):
#         df_mat.at[i,j] = df_mat[i][j] + b_time.T[j]

# df_mat.at[0,0]=df_mat[0][0] + b_time.T[0]
# df_mat

In [18]:
#pd.DataFrame(q_matrix.T.dot(p_matrix)).add(mu).head()

In [20]:
matrix_before_time =  (q_matrix.T.dot(p_matrix))+ mu + b_item.T + b_user
matrix_after_time = matrix_before_time.copy()

In [22]:
df_mat =  pd.DataFrame(matrix_after_time)

In [23]:
matrix_after_time=matrix_after_time.astype(object)
df_mat =  pd.DataFrame(matrix_after_time)
df_mat.at[0,0]=df_mat[0][0] + b_time.T[0]
df_mat

,0,1,2,3,4,5,6,7,8,9,...,600,601,602,603,604,605,606,607,608,609
0,"[4.676843591646274, 4.6875362453183715, 3.9326...",3.77226,3.95158,3.24824,3.97677,3.78313,3.89872,3.43824,3.5676,3.19889,...,4.1946,3.83141,3.55245,3.56105,3.51719,3.68796,4.04467,3.3798,3.15103,3.78232
1,4.04786,2.88755,3.49035,2.99371,3.40027,3.35659,2.84053,3.71174,3.31098,3.01566,...,3.73955,3.44857,3.06444,3.33386,3.20786,3.35724,3.44079,3.21005,2.98787,3.47651
2,3.84905,3.67159,3.26535,2.91515,3.18916,3.39217,3.46351,2.81372,2.59617,3.65587,...,3.79701,3.01547,3.33085,3.24617,3.2391,3.32554,3.41449,2.98416,3.10864,3.38482
3,3.40098,2.90188,3.28496,2.70219,2.7202,2.42964,2.97118,3.18441,2.13634,1.12589,...,3.0311,3.29355,2.65819,2.35302,2.13839,2.5634,2.62835,1.88767,2.27406,2.66989
4,3.89668,2.69567,3.04535,2.67681,3.40329,3.32746,2.90555,3.24296,3.27562,3.44798,...,3.48262,3.01396,2.95666,3.26675,2.93407,3.17893,3.18519,2.99324,3.11914,3.32869
5,4.51189,3.71151,2.28175,3.82626,3.57279,4.04999,3.23553,3.68786,4.03535,3.51153,...,4.30696,3.76631,3.51148,4.0727,3.69373,3.83047,4.21073,3.65556,3.49835,3.94939
6,3.96329,3.32028,2.50089,3.19837,2.88933,3.4429,2.84818,3.21074,3.05591,3.00115,...,3.8316,3.24834,3.26329,3.34147,3.1298,3.32917,3.40057,3.02633,2.97177,3.4602
7,3.82421,3.32406,2.57405,3.33015,3.18797,3.3132,3.17662,3.19806,1.96378,2.56882,...,4.11356,3.68744,3.43519,3.47424,2.62528,3.38047,4.04799,2.71727,3.49722,3.17664
8,3.7916,3.21744,1.86216,3.05891,3.28623,3.29486,3.2009,2.69035,3.47602,2.85027,...,3.2628,3.03657,2.89843,3.13098,2.82056,2.97516,3.26531,2.6287,2.79736,3.17959
9,4.23554,3.4107,2.30626,3.25667,3.26107,3.86859,3.09694,3.32176,3.89932,3.91387,...,3.93875,3.17694,3.24383,3.88103,3.3602,3.56838,3.58581,3.41386,3.54754,3.71977


In [126]:
d = cosine_similarity(test_movieid_given_user_str[0],1)

In [127]:
d

[('1445', 0.592428177293655),
 ('3441', 0.5238184005117847),
 ('2137', 0.41007026068952807),
 ('2116', 0.370094205662481),
 ('3617', 0.3225903098519638),
 ('70', 0.31945076983818543),
 ('1777', 0.30646796985042385),
 ('1732', 0.30272091575464966),
 ('2502', 0.2886788288660969),
 ('2058', 0.2853007337903877),
 ('1473', 0.27563443996655257),
 ('2414', 0.2561078909444748),
 ('2654', 0.2527580178715182),
 ('2000', 0.2509632060976217),
 ('1029', 0.250280159014815),
 ('1025', 0.2458422204667194),
 ('223', 0.23402350976504907),
 ('1214', 0.2312097934821),
 ('2253', 0.2278804860347231),
 ('1298', 0.21881291017644047),
 ('2366', 0.21394938898080385),
 ('1073', 0.20924968179619574),
 ('235', 0.20869612789221678),
 ('673', 0.20823004901446113),
 ('1097', 0.20488626053999656),
 ('1127', 0.20242446593744576),
 ('2826', 0.19971607485135823),
 ('736', 0.19646595567173789),
 ('1348', 0.1951615061997804),
 ('590', 0.19146261511729604),
 ('2450', 0.1874394286409211),
 ('1080', 0.18554934993213865),
 ('2

In [79]:
l_latent_movie = []
for movie in train_movieid_given_user_str:
    #print(movie,type(movie)) #2，str
    
    
    
    numerator =array([val
                  for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
                  for val in sublist]).dot(array([val
                  for sublist in np.asarray(q[[movie]]).tolist()
                  for val in sublist]))
    
    cosin_s = numerator/(norm([val
                  for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
                  for val in sublist])*norm([val
                  for sublist in np.asarray(q[[i]]).tolist()
                  for val in sublist]))
    l_latent_movie.append(cosin_s)
    
index_min = np.argmin(l_latent_movie) #find the index of min value

In [80]:
index_min

102

In [58]:
# movie_index = list(q.columns[1:]) #list of all movie index,str
# movie_index_int = [int(val) for val in movie_index] #int form of movie_index


# Vj_index = ['1'] # list of given movie index
# movie_index_sub = [item for item in movie_index if item not in Vj_index] #list of movie indexs except the comparied one

In [71]:
Vj_index = ['1'] # list of given movie index

movie_index_sub = [item for item in movie_index if item not in Vj_index] #list of movie indexs except the comparied one

In [86]:
# Vj_index = test_movieid_given_user_str
# Vj_index[0] #要predict 的电影ID

In [ ]:
l_latent_movie = []
for i in movie_index_sub:
   # print(i,type(i)) 2，str

#     numerator =array([val
#                   for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
#                   for val in sublist]).dot(array([val
#                   for sublist in np.asarray(q[[i]]).tolist()
#                   for val in sublist]))
    
#     cosin_s = numerator/(norm([val
#                   for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
#                   for val in sublist])*norm([val
#                   for sublist in np.asarray(q[[i]]).tolist()
#                   for val in sublist]))
#     l_latent_movie.append(cosin_s)
    
# index_min = np.argmin(l_latent_movie) #find the index of min value

# if int(Vj_index[0]) > index_min: 
#     index_similar = index_min
# else:
#     index_similar = index_min+1 #+1 would be the real index of that similar movie
    
# print(index_similar)

In [54]:
#q.as_matrix(columns=q.columns[1:2]).dot(q.as_matrix(columns=q.columns[2:3]))

In [123]:
l_latent_movie = []
movie_index = q.columns[1:]
# for i in range(len(q.columns[1:])):
#     print(array([val
#                   for sublist in np.asarray(q[[movie_index[i+1]]]).tolist()
#                   for val in sublist]).dot(array([val
#                   for sublist in np.asarray(q[[movie_index[i+2]]]).tolist()
#                   for val in sublist])))
#     #j = i+1
#     l_latent_movie.append(array([val
#                   for sublist in np.asarray(q[[movie_index[i+1]]]).tolist()
#                   for val in sublist]).dot(array([val
#                   for sublist in np.asarray(q[[movie_index[i+2]]]).tolist()
#                   for val in sublist])))

In [4]:
rmse.head()

,Unnamed: 0,train_test,par,epoch,RMSE
0,1,Train,"f = 10 , lambda = 0.01",1,0.825492
1,2,Train,"f = 20 , lambda = 0.01",1,0.792257
2,3,Train,"f = 10 , lambda = 0.1",1,0.896950
3,4,Train,"f = 20 , lambda = 0.1",1,0.859586
4,5,Test,"f = 10 , lambda = 0.01",1,1.313171


In [8]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]

In [13]:
neigh = NearestNeighbors(2, 0.4,metric='cosine')
neigh.fit(samples)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                 radius=0.4)

In [14]:
#class sklearn.neighbors.NearestNeighbors(n_neighbors=5, radius=1.0, algorithm=’auto’, leaf_size=30,
#metric=’minkowski’, p=2, metric_params=None, n_jobs=None, **kwargs)

In [15]:
neigh.predict([0,0,5])

AttributeError: 'NearestNeighbors' object has no attribute 'predict'